In [1]:
import requests
import json 
import pandas as pd

In [2]:
# sanity check, data verification

#since the data that we extracted is cumulative we need to take the difference
#meaning if say each day has 1 case, the third day value is 3 and not 1
#hence, we need to transform daily basis and not cumulative

url = "https://api.covid19api.com/total/country/singapore/status/confirmed"

response = requests.request("GET", url)
parsed_data = json.loads(response.text)
temp = pd.DataFrame.from_dict(parsed_data)
temp.set_index('Date')
temp['Cases'] = temp['Cases'].diff()
temp.Cases.fillna(0).sum()

# We can that verify with worldometer that is figures are pretty close at 64,981 
# https://www.worldometers.info/coronavirus/country/singapore/

64861.0

In [3]:
# we're going to build a dashboard for ASEAN instead of just SG
sea = ['brunei','cambodia','indonesia','laos','malaysia','myanmar','philippines','singapore','thailand','vietnam']

rename_country ={
'Brunei Darussalam':'Brunei',
'Lao PDR':'Laos',
'Viet Nam':'Vietnam'
}
rename_list = ['brunei','laos','vietnam']

url_pattern = 'https://api.covid19api.com/total/country/{}/status/confirmed'
required_cols =['Country','Cases','Date']

df = pd.DataFrame()
for country in sea:
    # scrape data from api based on country and concat accordingly
    # to form a dataset
    url = url_pattern.format(country)
    response = requests.request("GET", url)
    parsed_data = json.loads(response.text)
    temp = pd.DataFrame.from_dict(parsed_data)
    
    temp=temp[required_cols] # we only want columns we cared for
    
    #since the data that we scraped is cumulative we need to take the difference
    #meaning if say each day has 1 case, the third day value is 3 and not 1
    temp['Cases'] = temp['Cases'].diff()
    temp.Cases.fillna(0) #since the first row is null as it's shifted by 'diff'
    
    if country in rename_list: # cleaning data by changing names
        temp['Country'] = temp['Country'].map(rename_country)
        
    df = pd.concat([df,temp])

In [4]:
#extract more resolution from date_time
df['year'] = pd.DatetimeIndex(df['Date']).year
df['month'] = pd.DatetimeIndex(df['Date']).month
df['day'] = pd.DatetimeIndex(df['Date']).day
df['quarter'] = pd.DatetimeIndex(df['Date']).quarter
df['date'] = pd.DatetimeIndex(df['Date'])
df['year_month'] = pd.DatetimeIndex(df['Date']).strftime('%Y-%m')

# delete the not so useful column
del df['Date']

In [5]:
# samples 10 rows of data
df.sample(10)

,Country,Cases,year,month,day,quarter,date,year_month
476,Vietnam,87.0,2021,5,12,2,2021-05-12 00:00:00+00:00,2021-05
21,Laos,0.0,2020,2,12,1,2020-02-12 00:00:00+00:00,2020-02
505,Cambodia,426.0,2021,6,10,2,2021-06-10 00:00:00+00:00,2021-06
281,Cambodia,1.0,2020,10,29,4,2020-10-29 00:00:00+00:00,2020-10
200,Vietnam,29.0,2020,8,9,3,2020-08-09 00:00:00+00:00,2020-08
440,Thailand,250.0,2021,4,6,2,2021-04-06 00:00:00+00:00,2021-04
107,Malaysia,68.0,2020,5,8,2,2020-05-08 00:00:00+00:00,2020-05
421,Vietnam,3.0,2021,3,18,1,2021-03-18 00:00:00+00:00,2021-03
83,Myanmar,1.0,2020,4,14,2,2020-04-14 00:00:00+00:00,2020-04
403,Laos,0.0,2021,2,28,1,2021-02-28 00:00:00+00:00,2021-02


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5560 entries, 0 to 555
Data columns (total 8 columns):
Country       5560 non-null object
Cases         5550 non-null float64
year          5560 non-null int64
month         5560 non-null int64
day           5560 non-null int64
quarter       5560 non-null int64
date          5560 non-null datetime64[ns, UTC]
year_month    5560 non-null object
dtypes: datetime64[ns, UTC](1), float64(1), int64(4), object(2)
memory usage: 390.9+ KB


In [7]:
df.to_csv('covid.csv',index=False)